[View in Colaboratory](https://colab.research.google.com/github/tolany/Study-Python/blob/master/Data_Wrangling_with_python.ipynb)

 전문 프로그래머가 아닌 일반인이 프로그래밍 공부를 하는 이유는, 프로그래머가 되기 위해서라기 보다는 현재 하고 있는 일의 좋은 도구로서 프로그래밍을 고려하고 있기 때문일 것이다. 이 경우, 우리는 2가지 측면을 고려해야한다. 첫 번째는 학습의 난이도가 적당한지이다. 일의 능률을 위한 학습의 난이도가 너무 높다면, 주객이 전도되는 상황을 초래할 수 있기 때문이다. 두 번째는 실용적인지이다. 애써 학습을 해서 익혔는데, 막상 일에서 거의 쓸 일이 없다면 허무한 일이 아닐 수 없다. 
 
  이런 측면에서 파이썬, 그리고 이 책은 꽤 유용할 것이라고 생각한다. 우선 파이썬이라는 언어는 상대적으로 학습하고 다루기 쉬운 언어 중 하나이기 때문에 첫 번째 사항을 만족한다. 그리고 관련 라이브러리가 충분히 나와있는 상황이기 때문에 실제 업무에서 사용하는 툴과 연동해서 사용하기가 상당히 편리하다. 그리고 이 책은, 파이썬을 활용해서 거대한 프로그램을 설계하는 것에 대해 다른 전문적인 기술서라기 보다는 파이썬을 활용해서 실제 업무에 필요한 데이터를 핸들링하는 방법에 대해서 이야기하고 있는 책이다. 특히 실제 사용과 동 떨어진 예가 아니라 구체적이고, 현실의 업무에서 사용할 법한 데이터를 프로젝트 방식으로 다루면서 내용을 전개하고 있기 때문에 전문 프로그래머를 지망하는 것이 아닌 사람들에게 도움이 될 법한 책이라고 생각한다.
 

In [0]:
!apt update
!apt upgrade
!pip install xlrd
#!pip install slate
!pip install dataset
!pip install pdftables

In [0]:
from google.colab import files #Google Colab에 파일 업로드하기 

import sys, pprint 

import csv #3장 CSV File 다루기
import json #3장 JSON file 다루기 
from xml.etree import ElementTree as ET #3장 XML File 다루기

import xlrd #4장 excel file 다루기

#import slate # 5장 pdf file 다루기 
from pdftables import get_tables


import dataset # 6장 데이터 수집 및 저장하기(데이터 베이스)

from csv import DictReader # 7장 데이터 클리닝 기초 

## 2장 파이썬의 기초


### 2장 모두 종합하기. 



* 문자열, 리스트, 딕셔너리를 생성하라.


In [0]:
string = "example"
list = ["apple", "beer", "cat"]
diction = {}
diction["dog"] = 1
diction["cat"] = 2

* dir 매소드를 이용하여 각 데이터 유형에 사용할 수 있는 메소드를 검색하라 


In [0]:
dir(string)

In [0]:
dir(list)

In [0]:
dir(diction)

## 3장 기계가 읽을 수 있는 데이터 


### 3-1. CSV 

In [0]:
uploaded = files.upload()
csvfile = open('data-text.csv', 'rb')
reader = csv.reader(csvfile)

for row in reader:
  print row 

### 3-2 JSON


In [0]:
uploaded = files.upload()


In [0]:
json_data =open('data-text.json').read()
data = json.loads(json_data)

for i in data:
  print i

### 3-3 XML

In [0]:
uploaded = files.upload()

In [0]:
tree = ET.parse('data-text.xml')
root = tree.getroot()

data = root.find('Data')

all_data = []

for observation in data:
  record = {}
  
  for item in observation:
    
    lookup_key = item.attrib.keys()[0]
    
    if lookup_key == 'Numeric':
      rec_key = 'NUMERIC'
      rec_vale = item.attrib['Numeric']
    else:
      rec_key = item.attrib[lookup_key]
      rec_value = item.attrib['Code']
    
    record[rec_key] = rec_value 
  
  all_data.append(record)

print all_data

## 4. 엑셀 파일 다루기 

In [0]:
uploaded = files.upload()

In [0]:
book = xlrd.open_workbook('SOWC 2014 Stat Tables_Table 9.xlsx')

sheet = book.sheet_by_name('Table 9 ')

data = {}
for i in xrange(14, sheet.nrows): # 14열부터 시작, 데이터를 보면 14열부터 국가 데이터가 시작 
  row = sheet.row_values(i)
  country = row[1]
  
  data[country] = {
      'child_labor': {
          'total':[row[4], row[5]],
          'male':[row[6], row[7]],
          'female':[row[8], row[9]],
      },
      'child_marriage':{
          'married_by_15':[row[10], row[11]],
          'married_by_18':[row[12], row[13]],
      }
  }
  
  if country == 'Zimbabwe':
    break
pprint.pprint(data)

## 5장 PDF 다루기 

### 5-1 PDF Parse 

In [0]:
pdf = 'EN-FINAL Table 9.pdf'

with open(pdf) as f:
    doc = slate.PDF(f)

for page in doc[:2]:
    print type(page)


### 5-2 PDF Parse Text

In [0]:
pdf_txt = 'en-final-table9.txt'
openfile = open(pdf_txt, 'r')
country_line = total_line = False
previous_line = ''
countries = []
totals = []

double_lined_countries = [
    'Bolivia (Plurinational \n',
    'Democratic People\xe2\x80\x99s \n',
    'Democratic Republic \n',
    'Lao People\xe2\x80\x99s Democratic \n',
    'Micronesia (Federated \n',
    'Saint Vincent and \n',
    'The former Yugoslav \n',
    'United Republic \n',
    'Venezuela (Bolivarian \n',
]

def turn_on_off(line, status, start, prev_line, end='\n'):
 
    if line.startswith(start):
        status = True
    elif status:
        if line == end and prev_line != 'and areas':
            status = False
    return status


def clean(line):

    line = line.strip('\n').strip()
    line = line.replace('\xe2\x80\x93', '-')
    line = line.replace('\xe2\x80\x99', '\'')
    return line


for line in openfile:
    if country_line:
        if previous_line in double_lined_countries:
            line = ' '.join([clean(previous_line), clean(line)])
        countries.append(clean(line))

    elif total_line:
        if len(line.replace('\n', '').strip()) > 0:
            totals.append(clean(line))



    country_line = turn_on_off(line, country_line, 'and areas', previous_line)
    total_line = turn_on_off(line, total_line, 'total', previous_line)

    previous_line = line

data = dict(zip(countries, totals))
pprint.pprint(data)


### 5-3 PDF Table Table 

In [0]:
headers = ['Country', 'Child Labor 2005-2012 (%) total',
           'Child Labor 2005-2012 (%) male',
           'Child Labor 2005-2012 (%) female',
           'Child Marriage 2005-2012 (%) married by 15',
           'Child Marriage 2005-2012 (%) married by 18',
           'Birth registration 2005-2012 (%)',
           'Female Genital mutilation 2002-2012 (prevalence), women',
           'Female Genital mutilation 2002-2012 (prevalence), girls',
           'Female Genital mutilation 2002-2012 (support)',
           'Justification of wife beating 2005-2012 (%) male',
           'Justification of wife beating 2005-2012 (%) female',
           'Violent discipline 2005-2012 (%) total',
           'Violent discipline 2005-2012 (%) male',
           'Violent discipline 2005-2012 (%) female']

all_tables = get_tables(open('EN-FINAL Table 9.pdf', 'rb'))

first_name = False
final_data = []

for table in all_tables:
    for row in table[5:]:
        if row[0] == '' or row[0][0].isdigit():
            continue
        elif row[2] == '':
            first_name = row[0]
            continue
        if first_name:
            row[0] = u'{} {}'.format(first_name, row[0])
            first_name = False

        final_data.append(dict(zip(headers, row)))

        if row[0] == 'Zimbabwe':
                break

pprint.pprint(final_data)


## 6장 데이터 수집 및 저장하기 

### 6-0 백엔드 설치 (SQLite)
1. 홈페이지(http://sqlite.org/download.html)에서 SQLite 바이너리를 다운받는다.
2. 다운받은 바이너리 파일을 파이썬 스크립트가 있는 폴더로 이동시킨다.
3. "sqlite3 data_wrangling.db" 명령어를 입력한다. 

### 6-1 데이터베이스 

In [0]:
db = dataset.connect('sqlite:///data_wrangling.db')

my_data_source = {
    'url':
    'http://www.tsmplug.com/football/premier-league-player-salaries-club-by-club/',
    'description':'Premier League Club Salaries',
    'topic':'football',
    'verified':False,
}

table = db['data_sources']
table.insert(my_data_source)

another_data_source = {
    'url':
    'http://www.premierleague.com/content/premierleague/en-gb/players/index.html',
    'description': 'Premier League Stats',
    'topic':'football',
    'verified':True,
}

table.insert(another_data_source)
sources = db['data_sources'].all()
print sources 

## 7장 데이터 클리닝 기초 

In [0]:
uploaded = files.upload()

### 7-1 데이터 클리닝을 위한 값 찾기 

In [0]:
from csv import reader

data_rdr = reader(open('mn.csv', 'rb'))
header_rdr = reader(open('mn_headers.csv', 'rb'))

data_rows = [d for d in data_rdr]
header_rows = [h for h in header_rdr]

print len(data_rows[0])
print len(header_rows)


bad_rows = []
for h in header_rows:
    if h[0] not in data_rows[0]:
        bad_rows.append(h)

for h in bad_rows:
    header_rows.remove(h)

print len(header_rows)

all_short_headers = [h[0] for h in header_rows]
for header in data_rows[0]:
    if header not in all_short_headers:
        print 'mismatch!', header
